In [78]:
import os
import pdfplumber

data_folder = "E:\\Resume_Macher_Project\\data"
jd_pdf_path = os.path.join(data_folder, "jd.pdf")
with pdfplumber.open(jd_pdf_path) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text() + "\n"

resume_text = {}
for file in os.listdir(data_folder):
    if file.endswith(".pdf") and file != "jd.pdf":
        path = os.path.join(data_folder, file)
        resume_text[file] = "" 
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                resume_text[file] += page.extract_text() + "\n"
print("JD length:", len(text))
print("Total resumes extracted:", len(resume_text))

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


JD length: 1799
Total resumes extracted: 8


//Data extracted from pdf

In [79]:
import os
ouput_folder = "E:\\Resume_Macher_Project\\output"
os.makedirs(ouput_folder, exist_ok=True)

with open(os.path.join(ouput_folder, "jd.txt"), "w", encoding="utf-8") as f:
    f.write(text.strip())
    
for filename, text in resume_text.items():
    with open(os.path.join(ouput_folder, filename.replace(".pdf", ".txt")), "w", encoding="utf-8") as f:
        f.write(text.strip())
print("JD and resumes have been extracted and saved to the output folder.")

JD and resumes have been extracted and saved to the output folder.


//Extracted text converted and saved as txt file

In [80]:
input_folder = "E:\\Resume_Macher_Project\\output"
output_folder = "E:\\Resume_Macher_Project\\output\\cleaned"
os.makedirs(output_folder, exist_ok=True)

In [81]:
import re
import spacy
nlp = spacy.load("en_core_web_sm")
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    doc = nlp(text)
    cleaned = " ".join([token.text for token in doc if not token.is_stop])
    return cleaned
    

In [82]:
for file in os.listdir(input_folder):
    if file.endswith(".txt"):
        with open(os.path.join(input_folder, file), "r", encoding="utf-8") as f:
            raw_text = f.read()
        cleaned = clean_text(raw_text)
        with open(os.path.join(output_folder, file), "w", encoding="utf-8") as f:
            f.write(cleaned)
print("Text cleaning completed. Cleaned files are saved in the cleaned folder.")

Text cleaning completed. Cleaned files are saved in the cleaned folder.


//txt file cleaned and saved

In [83]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-base-en')
print("Model loaded successfully.")

Model loaded successfully.


In [84]:
import os
input_folder = "E:\\Resume_Macher_Project\\output\\cleaned"
text = {}
for file in os.listdir(input_folder):
    if file.endswith(".txt"):
        with open(os.path.join(input_folder, file), "r", encoding="utf-8") as f:
            text[file] = f.read()
print("Loaded:", len(text),"cleaned text files")

Loaded: 9 cleaned text files


In [85]:
embeddings = {}
for filename, content in text.items():
    embeddings[filename] = model.encode(content, convert_to_tensor=True)
    print("embeddings created")

embeddings created
embeddings created
embeddings created
embeddings created
embeddings created
embeddings created
embeddings created
embeddings created
embeddings created


In [86]:
from sentence_transformers import util
jd_file = "jd.txt"
jd_embedding = embeddings[jd_file]
similarity_scores = {}
for file, emb in embeddings.items():
    if file != jd_file:
        score = util.pytorch_cos_sim(jd_embedding, emb).item()
        similarity_scores[file] = round(score * 100, 2)
sorted_scores = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
print("Top Resume Matches:")
for file, score in sorted_scores:
    print(f"{file}:{score}%match")

Top Resume Matches:
Resume8.txt:91.72%match
Resume2.txt:89.19%match
Resume4.txt:87.16%match
Resume1.txt:83.99%match
Resume7.txt:83.02%match
Resume6.txt:82.55%match
Resume3.txt:81.15%match
Resume5.txt:76.48%match


In [87]:
import spacy 
import os
import re

nlp = spacy.load("en_core_web_sm")


def extract_keywords_from_jd(jd_text):
    job_title = ""
    skills = []
    experience = ""  
    lines = jd_text.split('\n')
    for line in lines:
        lower_line = line.lower()
        if 'title' in lower_line and not job_title:
            job_title = line.strip()
        if 'skills' in lower_line or 'requested' in lower_line:
            skills += re.findall(r'\b\w+\b', line.lower())
        if 'experience' in lower_line and not experience:
            experience = line.strip()
    return job_title.lower(), set(skills), experience.lower()

with open(os.path.join(input_folder, "jd.txt"), "r", encoding='utf-8') as f:
    jd_raw_text = f.read()

jd_title, jd_skills, jd_experience = extract_keywords_from_jd(jd_raw_text)


resume_text = {}
import os
data_folder = "E:\\Resume_Macher_Project\\data"
for filename in os.listdir(data_folder):
    if filename.endswith(".txt") and filename != "jd.txt":
        with open(os.path.join(data_folder, filename), 'r', encoding='utf-8') as f:
            resume_text[filename] = f.read()


matched_resumes = []
unmatched_resumes = []

for filename, resume in resume_text.items():
    text = resume.lower()
    skill_match = any(skill in text for skill in jd_skills)
    title_match = jd_title.split()[-1] in text if jd_title else False
    experience_match = jd_experience.split()[-1] in text if jd_experience else False

    if skill_match or title_match or experience_match:
        matched_resumes.append((filename, text))
    else:
        unmatched_resumes.append((filename, text))

print("Matched Resumes:")
for f in matched_resumes:
    print(f[0])

print("\n Unmatched Resumes:")
for f in unmatched_resumes:
    print(f[0])


                

Matched Resumes:

 Unmatched Resumes:


In [88]:
from sentence_transformers import util
def match_resumes(jd_text, resume_texts, model, weight_semantics=0.4, weight_keyword=0.6):
    jd_keywords = extract_keywords_from_jd(jd_text)
    jd_embedding = model.encode(jd_text, convert_to_tensor=True)
    results = []
    for filename, text in resume_texts.items():
        resume_embedding = model.encode(text, convert_to_tensor=True)
        semantic_score = util.pytorch_cos_sim(jd_embedding, resume_embedding).item() * 100
        matched_keywords = sum([1 for kw in jd_keywords if kw.lower() in text.lower()])
        keyword_score = (matched_keywords / len(jd_keywords)) * 100

        final_score = weight_semantics * semantic_score + weight_keyword * keyword_score

        results.append({
            "filename": filename,
            "semantic_score": round(semantic_score, 2),
            "keyword_score": round(keyword_score, 2),
            "final_score": round(final_score, 2)
        })

    return sorted(results, key=lambda x: x["final_score"], reverse=True)

In [89]:
jd_text = text["jd.txt"]
resumes = {k: v for k, v in text.items() if k != "jd.txt"}

results = match_resumes(jd_text, resumes, model)
for r in results:
    print(r)



AttributeError: 'set' object has no attribute 'lower'

In [ ]:
import pandas as pd
df_results = pd.DataFrame(results)
df_results.to_csv("matching_results.csv", index=False)
print("Results saved to matching_results.csv")

Results saved to matching_results.csv


In [ ]:
results = match_resumes(jd_text, resumes, model)
filtered_results = [r for r in results if r["final_score"] >= 75.0]
print("\n Resumes with final score >= 75%:\n")
for r in filtered_results:
    print(r)


 Resumes with final score >= 75%:

{'filename': 'Resume8.txt', 'semantic_score': 91.72, 'keyword_score': 70.0, 'final_score': 78.69}
{'filename': 'Resume2.txt', 'semantic_score': 89.19, 'keyword_score': 70.0, 'final_score': 77.68}


In [ ]:
import pandas as pd
df_filtered = pd.DataFrame(filtered_results)
df_filtered.to_csv("filtered_matching_results.csv", index=False)
print("Saved filtered results to filtered_matching_results.csv")


Saved filtered results to filtered_matching_results.csv
